#  Stink Bug Pipline

This notebook grab the city data and brown stink bug survey data from Minnesota. It then processes the data to conduct QAQC steps, which is to check the coordinate system and convert it to 4326 for POSTGIS, and identify duplicate cities. In the end, it creates a table that calculate distance between cities

In [1]:
import arcpy
import json
import zipfile
import pprint
import requests
from arcpy import env
import os
from zipfile import ZipFile
import random
import time
import pandas as pd
from dbfread import DBF
import numpy as np

In [48]:
# Function to download a file from a URL and unzip it
def get_data(url, local_directory):
    response = requests.get(url)
    file_name = os.path.join(local_directory, url.split("/")[-1])

    with open(file_name, "wb") as f:
        f.write(response.content)
        
    with ZipFile(file_name, "r") as zip_ref:
        zip_ref.extractall(local_directory)

# Define local directory
local_directory = r"E:\coursework\ARCGIS2\BMSMProject\data"
# Create the local directory if it doesn't exist
if not os.path.exists(local_directory):
    os.makedirs(local_directory)

# Define the URL
boundary_url = "https://resources.gisdata.mn.gov/pub/gdrs/data/pub/us_mn_state_dot/bdry_mn_city_township_unorg/shp_bdry_mn_city_township_unorg.zip"
bmsb_url = "https://resources.gisdata.mn.gov/pub/gdrs/data/pub/us_mn_state_mda/biota_bmsb/shp_biota_bmsb.zip"

get_data(boundary_url,local_directory)
get_data(bmsb_url,local_directory)

print('Data extraction is done!')

Data extraction is done!


## QAQC 

first step of the qaqc is to create a new column "ObsNum" that sums the "Adults" and "Nymphs" column together, and then create a "ground trugh" column based on "ObsNum" column

Second step is to convert the city shapefile to 4326, select only cities and remove unnecesary fields and identify duplicate cities by sum their populatino and merge the duplicate cities polygons.

Third step is to join the BMSB table to MN_Cities data and find the closest city to BMSB.


### BMSB data QAQC

In [53]:
# Convert BMSB table to pandas df
bmsb_dbf = DBF(os.path.join(local_directory,'BMSBSurveyDataTable.dbf'))

bmsb_df = pd.DataFrame(iter(bmsb_dbf))

#clean and format data
bmsb_df = bmsb_df[["SiteName", "City", "Latitude", "Longitude", "Year",'CheckDate','Adults','Nymphs']].copy()
bmsb_df = bmsb_df.dropna(subset=["Latitude", "Longitude", 'CheckDate','Adults','Nymphs'])
bmsb_df["CheckDate"] = bmsb_df["CheckDate"].astype('datetime64[ns]')
bmsb_df["month"] = pd.DatetimeIndex(bmsb_df["CheckDate"]).month

#bounding box
bmsb_df = bmsb_df[(bmsb_df["Longitude"] > -97.239) & (bmsb_df["Longitude"] < -89.492) &
                              (bmsb_df["Latitude"] > 43.499) & (bmsb_df["Latitude"] < 49.384)]

#calculate bmsb_dfnum of obs
bmsb_df['ObsNum'] = bmsb_df['Adults'] + bmsb_df['Nymphs']
#bmsb_df['Presence'] = 0
#bmsb_df.loc[bmsb_df['ObsNum']>0,'Presence'] = 1

bmsb_df = bmsb_df[bmsb_df['ObsNum']>0]

bmsb_df

,SiteName,City,Latitude,Longitude,Year,CheckDate,Adults,Nymphs,month,ObsNum
55,AFTON APPLE ORCHARD,HASTINGS,44.818630,-92.813420,2015.0,2015-09-23,1.0,0.0,9,1.0
122,AFTON APPLE ORCHARD,HASTINGS,44.818870,-92.813330,2018.0,2018-09-27,1.0,0.0,9,1.0
133,AFTON APPLE ORCHARD,HASTINGS,44.818654,-92.813416,2019.0,2019-09-25,3.0,0.0,9,3.0
197,ALEXIS BAILLY VINEYARD,HASTINGS,44.686010,-92.871800,2020.0,2020-09-10,1.0,0.0,9,1.0
548,BATTLE CREEK,SAINT PAUL,44.916890,-93.010860,2016.0,2016-10-12,58.0,3.0,10,61.0
...,...,...,...,...,...,...,...,...,...,...
7779,NE RIVERSIDE GARDEN CLUB,MINNEAPOLIS,45.023028,-93.271555,2021.0,2021-08-16,1.0,0.0,8,1.0
7780,NE RIVERSIDE GARDEN CLUB,MINNEAPOLIS,45.022958,-93.271704,2021.0,2021-09-10,5.0,1.0,9,6.0
7790,MERRIAM STATION COMMUNITY GARDEN,ST. PAUL,44.954310,-93.184555,2021.0,2021-08-16,1.0,0.0,8,1.0
7791,NE RIVERSIDE GARDEN CLUB,MINNEAPOLIS,45.022971,-93.271885,2021.0,2021-09-01,4.0,0.0,9,4.0


In [54]:
arcpy.env.workspace = "E:\coursework\ARCGIS2\BMSMProject\data"

sr = arcpy.SpatialReference(4326) # WGS 1984

# create a new feature class
arcpy.CreateFeatureclass_management(arcpy.env.workspace, "bmsb", "POINT",spatial_reference=sr)

# add fields to the feature class
arcpy.AddField_management("bmsb.shp", "SiteName", "Text")
arcpy.AddField_management("bmsb.shp", "City", "Text")
arcpy.AddField_management("bmsb.shp", "Latitude", "Double")
arcpy.AddField_management("bmsb.shp", "Longitude", 'Double')
arcpy.AddField_management("bmsb.shp", "CheckDate", 'Date')
arcpy.AddField_management("bmsb.shp", "month", "Short")
arcpy.AddField_management("bmsb.shp", "Year", "Short")
arcpy.AddField_management("bmsb.shp", "ObsNum", "Short")
#arcpy.AddField_management("bmsb.shp", "Presence", "Short")

# insert data into the feature class
cursor = arcpy.da.InsertCursor("bmsb.shp", ["SHAPE@", "SiteName", "City","Latitude",'Longitude','CheckDate','month','Year','ObsNum'])
for index,row in bmsb_df.iterrows():
    point = arcpy.Point(row[3], row[2])
    cursor.insertRow([point, row[0], row[1],row[2], row[3],row[5], row[8],row[4], row[9]])
del cursor

### City data QAQC

In [85]:
#saves city polygons
#expression = 
arcpy.conversion.FeatureClassToFeatureClass("city_township_unorg", 
                                            "E:\coursework\ARCGIS2\BMSMProject\data", 
                                            "City.shp", "CTU_CLASS = 'CITY'")

#make sure the spatial reference system is 4326
sr = arcpy.SpatialReference(4326) # WGS 1984
arcpy.management.Project("City.shp", r"City_reproject.shp", sr)

#merge cities with the same name
arcpy.management.Dissolve("City_reproject", "City_qaqc_done", "FEATURE_NA", [["POPULATION", 'SUM']])

# Alter name FEATURE_NA to CITY_NAME
arcpy.management.CalculateField("City_qaqc_done", "CITY_NAME", "!FEATURE_NA!")
arcpy.management.CalculateField("City_qaqc_done", "POPULATION", "!SUM_POPULA!",field_type = 'LONG')

#delete unnecessary columns
arcpy.management.DeleteField("City_qaqc_done", "CITY_NAME;POPULATION", "KEEP_FIELDS")

#convert each City to points (has been poylgons until now)
arcpy.management.FeatureToPoint("City_qaqc_done", "City_Point.shp", "CENTROID")
arcpy.management.DeleteField("City_Point", "CITY_NAME;POPULATION", "KEEP_FIELDS")

#simulation city shp layer
arcpy.CopyFeatures_management("City_Point", "City_Sim")
#prediction city shp layer
arcpy.CopyFeatures_management("City_Point", "City_Pred2022")
arcpy.CopyFeatures_management("City_Point", "City_Pred2023")
arcpy.CopyFeatures_management("City_Point", "City_Pred2024")

<Result 'E:\\coursework\\ARCGIS2\\BMSMProject\\data\\City_Point.shp'>

### Generate City FROM-TO table

In [80]:
#generate from-to table
arcpy.analysis.GenerateNearTable("City_Point", "City_Point", "E:\coursework\ARCGIS2\BMSMProject\Default.gdb\City_FromToTable", closest = "ALL", method = "GEODESIC")

<Result 'E:\\coursework\\ARCGIS2\\BMSMProject\\Default.gdb\\City_FromToTable'>

In [81]:
arcpy.env.workspace = "E:\coursework\ARCGIS2\BMSMProject\Default.gdb"

#delete unnecessary columns
arcpy.management.DeleteField("City_FromToTable", "IN_FID;NEAR_FID;NEAR_DIST", "KEEP_FIELDS")

#format from-to city and population
arcpy.management.JoinField("City_FromToTable", "IN_FID", "City_qaqc_done", "FID", ["CITY_NAME",'POPULATION'])
arcpy.management.AlterField("City_FromToTable", "CITY_NAME", 'FROM_CITY','FROM_CITY')
arcpy.management.AlterField("City_FromToTable", "POPULATION", 'FROM_POP','FROM_POP')
arcpy.management.JoinField("City_FromToTable", "NEAR_FID", "City_qaqc_done", "FID", ["CITY_NAME",'POPULATION'])
arcpy.management.AlterField("City_FromToTable", "CITY_NAME", 'TO_CITY','TO_CITY')
arcpy.management.AlterField("City_FromToTable", "POPULATION", 'TO_POP','TO_POP')

<Result 'City_FromToTable'>

# Land Cover Data QAQC

In [ ]:
#read data
raster_data = rasterio.open(landcover_path)

# Plot raster data
plt.imshow(raster_data.read(1), cmap='viridis')
plt.title('Land Cover Raster')
plt.colorbar()
plt.show()



In [ ]:
#bounding box
arcpy.management.Clip(landcover_path, "132660 4774410 791819 5491608", os.path.join(out_local, "lc_final"));

In [ ]:
#check if raster is within bounding box
bounding_box = (132660, 4774410, 791819, 5491608)


raster_file = r"C:\Users\Maochuan\OneDrive\桌面\lab2\NLCD_2019_Land_Cover.tif"
with rasterio.open(raster_file) as src:
    # Get the raster bounding box
    raster_bbox = src.bounds
    
# Check if the raster is within the bounding box
    is_within_bbox = (
        raster_bbox.left >= bounding_box[0] and
        raster_bbox.bottom >= bounding_box[1] and
        raster_bbox.right <= bounding_box[2] and
        raster_bbox.top <= bounding_box[3]
    )

# Print the result
if is_within_bbox:
    print("The raster is within the bounding box.")
else:
    print("The raster is not within the bounding box.")

In [ ]:
# Check if the data is categorical or not

# Set the workspace environment
arcpy.env.workspace = r"C:\Users\Maochuan\OneDrive\文档\ArcGIS\Projects\lab2_arc2"

# Define the land cover raster dataset
land_cover_tif = r"C:\Users\Maochuan\OneDrive\桌面\lab2\NLCD_2019_Land_Cover.tif"

# Conver the raster to numpy array
land_cover_array = arcpy.RasterToNumPyArray(land_cover_tif)

# Calculate the unique values in an array
unique_values = np.unique(land_cover_array)

# Set a threshold for the number of unique values to consider a raster categorical
threshold = 100

if len(unique_values) <= threshold:
    print("The land cover GeoTIFF is likely categorical.")
else:
    print("The land cover GeoTIFF is not categorical.")

# In this approach, we first convert the raster to a NumPy array
# then calculate the number of unique values in the array. 
# If the number of unique values is below a certain threshold (e.g., 100), 
# we consider the raster to be categorical. This is not a definitive method, 
# as the threshold value is arbitrary and could vary depending on your specific dataset. 

In [ ]:
# Check for null values
null_values = arcpy.management.GetRasterProperties(landcover_path, "ANYNODATA").getOutput(0)
if null_values == "1":
        print("Null values exist.")
else:
        print("Null values do not exist.")

In [ ]:
#coordinate system
with raster_data as src:
    raster_crs = src.crs

    print("Raster CRS:", raster_crs)

In [ ]:
#cell size

with raster_data as src1:
    # Get the cell size (resolution) from the transform object
    cell_size_x = src1.transform[0]
    cell_size_y = -src1.transform[4]

# Print the cell size
print(f"Cell size (X): {cell_size_x} units")
print(f"Cell size (Y): {cell_size_y} units")

# Landcover data extraction

In [ ]:
# clip landcover data to each city polygon
out_raster = arcpy.sa.ExtractByMask("NLCD_2019_Land_Cover.tif", "MN_Cities_dissolve_Project", "INSIDE", '157274.260712674 4816295.86531197 717221.411589083 5433614.51137255 PROJCS["NAD_1983_UTM_Zone_15N",GEOGCS["GCS_North_American_1983",DATUM["D_North_American_1983",SPHEROID["GRS_1980",6378137.0,298.257222101]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]],PROJECTION["Transverse_Mercator"],PARAMETER["False_Easting",500000.0],PARAMETER["False_Northing",0.0],PARAMETER["Central_Meridian",-93.0],PARAMETER["Scale_Factor",0.9996],PARAMETER["Latitude_Of_Origin",0.0],UNIT["Meter",1.0]]'); out_raster.save(r"C:\Users\Maochuan\OneDrive\文档\ArcGIS\Projects\arc2_final_project\arc2_final_project.gdb\Extract_NLCD1")

In [ ]:
# Convert raster to points with the pixel count value for each point and land class
arcpy.conversion.RasterToPoint("Extract_NLCD1", r"C:\Users\Maochuan\OneDrive\文档\ArcGIS\Projects\arc2_final_project\arc2_final_project.gdb\RasterT_Extract3", "Count")

In [ ]:
# Spatial Join the points to city polygon shapefile
arcpy.analysis.SpatialJoin("MN_Cities_dissolve_Project", "RasterT_Extract3", r"C:\Users\Maochuan\OneDrive\文档\ArcGIS\Projects\arc2_final_project\arc2_final_project.gdb\MN_Cities_dissol_SpatialJoin1", "JOIN_ONE_TO_ONE", "KEEP_ALL", 'CITY_SJ "CITY_SJ" true true false 512 Text 0 0,First,#,MN_Cities_dissolve_Project,CITY_SJ,0,512;Shape_Length "Shape_Length" false true true 8 Double 0 0,First,#,MN_Cities_dissolve_Project,Shape_Length,-1,-1;Shape_Area "Shape_Area" false true true 8 Double 0 0,First,#,MN_Cities_dissolve_Project,Shape_Area,-1,-1;POPULATION "POPULATION" true true false 4 Long 0 0,First,#,MN_Cities_dissolve_Project,POPULATION,-1,-1;pointid "pointid" true true false 4 Long 0 0,First,#,RasterT_Extract3,pointid,-1,-1;grid_code "grid_code" true true false 4 Long 0 0,First,#,RasterT_Extract3,grid_code,-1,-1;Name "Name" true true false 28 Text 0 0,First,#,RasterT_Extract3,Name,0,28', "INTERSECT", None, '')

In [ ]:
# calculate the total count of pixels by city
arcpy.Statistics_analysis("RasterT_Extract3_SpatialJoin", "output_table",[["Name", "COUNT"]], "CITY_SJ")

In [ ]:
input_feature_class = "RasterT_Extract3_SpatialJoin"
output_filtered_fc = "filtered_feature_class"
output_table = "output_table_agriculture"


attributes_to_match = [
    "Open Water",
    "Barren Land (Rock/Sand/Clay)",
    "Deciduous Forest",
    "Evergreen forest",
    "Mixed forest",
    "shrub/scrub",
    "Grassland/Herbaceous",
    "Pasture/Hay",
    "Cultivated Crops",
    "Woody Wetlands",
    "Emergent Herbaceous Wetlands"
]


# Make a feature layer from the input feature class
arcpy.MakeFeatureLayer_management(input_feature_class, "input_layer")

# Use a SQL expression to filter the layer based on the desired attributes
sql_expression = f"{arcpy.AddFieldDelimiters('input_layer', 'Name')} IN {tuple(attributes_to_match)}"
arcpy.SelectLayerByAttribute_management("input_layer", "NEW_SELECTION", sql_expression)

# Save the filtered layer to a new feature class
arcpy.CopyFeatures_management("input_layer", output_filtered_fc)

# Clean up the feature layer
arcpy.Delete_management("input_layer")

# calculate the total pixel count by each city by its agricultural pixel count
arcpy.Statistics_analysis(output_filtered_fc, output_table, [["Name", "COUNT"]], "CITY_SJ")

#add field for ag_percentage
arcpy.management.AddField("output_table_ag+city", "ag_percentage", "DOUBLE", None, None, None, '', "NULLABLE", "NON_REQUIRED", '')

# calculate the ag_percentage by dividng the agricultural field by total pixel count
arcpy.management.CalculateField("output_table_ag+city", "ag_percentage", "!FREQUENCY! / !FREQUENCY_1!", "PYTHON3", '', "TEXT", "NO_ENFORCE_DOMAINS")

# join Ag table

In [ ]:
arcpy.JoinField_management("City_FromToTable", "TO_CITY", "ag_percentage_to.csv", "CITY_TO")
arcpy.TableToTable_conversion("City_FromToTable", r"E:\coursework\ARCGIS2\BMSMProject\Default.gdb", "City_FromToTable_Final")

### Monte Carlo Simulation

In [35]:
def simulation(FromToTable,SimLayer,method,seed,num_year,G,i):
    PresenceCity = [seed]
    for j in range(0, num_year):
        with arcpy.da.SearchCursor(FromToTable,"*") as cursor:
            for row in cursor:
                if method == 'MC':
                    if row[4] in PresenceCity and row[6] not in PresenceCity and row[3]<=distance_threhold:
                        prob = max(0, random.normalvariate(0.05, 0.1 * 0.05))
                        if random.random()<= prob:
                            PresenceCity.append(row[6])
                elif method == 'G':
                    if row[4] in PresenceCity and row[6] not in PresenceCity:
                        if random.random()<= G * row[10]:
                            PresenceCity.append(row[6])
                elif method == 'H':
                    if row[4] in PresenceCity and row[6] not in PresenceCity:
                        if random.random()<= G * row[11]:
                            PresenceCity.append(row[6])
                else:
                    print('Please select methods among MC/G/H')
                    break

    print(f"Simulation{i} for Year{j+2016}:{PresenceCity}")

    # adds presence column for each simulation
    arcpy.management.AddField(SimLayer, f"Sim{i}_MC", "SHORT")
    with arcpy.da.UpdateCursor(SimLayer,"*") as cursor:
        for row in cursor:
            if method == 'MC':
                if row[2] in PresenceCity:
                    row[(4+i)] = 1
                else:
                    row[(4+i)] = 0
                cursor.updateRow(row)
            elif method == 'G':
                if row[2] in PresenceCity:
                    row[(5+i)] = 1
                else:
                    row[(5+i)] = 0
                cursor.updateRow(row)
            elif method == 'H':
                if row[2] in PresenceCity:
                    row[(6+i)] = 1
                else:
                    row[(6+i)] = 0
                cursor.updateRow(row)
    return PresenceCity

In [37]:
def prediction(FromToTable,PredLayer,method,PresenceCity,G,i):
    with arcpy.da.SearchCursor(FromToTable,"*") as cursor:
        for row in cursor:
            if method == 'MC':
                if row[4] in PresenceCity and row[6] not in PresenceCity and row[3]<=distance_threhold:
                    prob = max(0, random.normalvariate(0.05, 0.1 * 0.05))
                    if random.random()<= prob:
                        PresenceCity.append(row[6])
            elif method == 'G':
                if row[4] in PresenceCity and row[6] not in PresenceCity:
                    if random.random()<= G * row[10]:
                        PresenceCity.append(row[6])
            elif method == 'H':
                if row[4] in PresenceCity and row[6] not in PresenceCity:
                    if random.random()<= G * row[11]:
                        PresenceCity.append(row[6])
            else:
                print('Please select methods among MC/G/H')
    
    # adds presence column for each simulation
    arcpy.management.AddField(PredLayer, f"Pred{i}_MC", "SHORT")
    with arcpy.da.UpdateCursor(PredLayer,"*") as cursor:
        for row in cursor:
            if method == 'MC':
                if row[2] in PresenceCity:
                    row[(4+i)] = 1
                else:
                    row[(4+i)] = 0
                cursor.updateRow(row)
            elif method == 'G':
                if row[2] in PresenceCity:
                    row[(5+i)] = 1
                else:
                    row[(5+i)] = 0
                cursor.updateRow(row)
            elif method == 'H':
                if row[2] in PresenceCity:
                    row[(6+i)] = 1
                else:
                    row[(6+i)] = 0
                cursor.updateRow(row)
    return PresenceCity

In [36]:
def final_result(Layer,method):
    with arcpy.da.UpdateCursor(Layer,"*") as cursor:
        for row in cursor:
            if method == 'MC':
                if sum(row[4:54])>25:
                    row[54] = 1
                else:
                    row[54] = 0
                cursor.updateRow(row)
            elif method == 'G':
                if sum(row[5:55])>25:
                    row[55] = 1
                else:
                    row[55] = 0
                cursor.updateRow(row)
            elif method == 'H':
                if sum(row[6:56])>25:
                    row[56] = 1
                else:
                    row[56] = 0
                cursor.updateRow(row)

In [38]:
# Read values from field
distance_values = []
with arcpy.da.SearchCursor("City_FromToTable_Final", ["NEAR_DIST"]) as cursor:
    for row in cursor:
        if row[0] is not None:
            distance_values.append(row[0])

# Calculate the 1st quantile
distance_threhold = np.quantile(distance_values, 0.005)

for i in range(0,50):
    PresenceCity = simulation("City_FromToTable_Final","City_Sim",method = 'MC',
                              seed = 'Minneapolis',num_year = 6,G = None,i = i)
    
    PresenceCity = prediction("City_FromToTable_Final","City_Pred2022",method = 'MC',PresenceCity = PresenceCity, G = None,i = i)
    PresenceCity = prediction("City_FromToTable_Final","City_Pred2023",method = 'MC',PresenceCity = PresenceCity, G = None,i = i)
    PresenceCity = prediction("City_FromToTable_Final","City_Pred2024",method = 'MC',PresenceCity = PresenceCity, G = None,i = i)

Simulation0 for Year2021:['Minneapolis', 'Mendota', 'Bloomington', 'Hopkins', 'Savage', 'Eden Prairie', 'Edina', 'Greenwood', 'Minnetonka', 'Medicine Lake', 'Excelsior', 'Golden Valley', 'Woodland', 'Deephaven', 'Lilydale', 'Richfield', 'Orono', 'Minnetonka Beach', 'Mound', 'Spring Park', 'Wayzata', 'Lauderdale', 'Saint Louis Park', 'Maple Plain', 'Independence', 'Credit River', 'Fridley', 'Watertown', 'Crystal', 'Medina', 'Brooklyn Center', 'South Saint Paul', 'Loretto', 'Inver Grove Heights', 'Waconia']
Simulation1 for Year2021:['Minneapolis', 'Falcon Heights', 'Columbia Heights', 'New Hope', 'Gem Lake', 'White Bear Lake', 'Mahtomedi', 'Dellwood', 'Maplewood', 'Robbinsdale', 'Mendota', 'Fridley', 'Willernie', 'Little Canada', 'Lake Elmo', 'Shoreview', 'Golden Valley', 'Hopkins', 'Woodland', 'Wayzata', 'Lake Saint Croix Beach', 'Birchwood Village', 'Grant', 'Woodbury', 'Sunfish Lake', 'South Saint Paul', 'Hilltop', 'Plymouth', 'Osseo', 'Vadnais Heights', 'Inver Grove Heights', 'Greenw

Simulation22 for Year2021:['Minneapolis', 'Robbinsdale', 'Medicine Lake', 'Spring Lake Park', 'Brooklyn Park', 'Brooklyn Center', 'Golden Valley', 'Hilltop', 'New Hope', 'Lauderdale', 'Maple Grove', 'Plymouth', 'Arden Hills', 'Columbia Heights', 'Wayzata', 'Crystal', 'Champlin', 'Hopkins', 'New Brighton', 'Saint Louis Park', 'Saint Paul', 'Richfield', 'Mounds View', 'Minnetonka', 'Shorewood', 'White Bear Lake', 'Woodland', 'Edina', 'Osseo', 'Fridley', 'Deephaven', 'Greenwood', 'Vadnais Heights', 'Hugo', 'Lino Lakes']
Simulation23 for Year2021:['Minneapolis', 'New Brighton', 'Lexington', 'Circle Pines', 'Brooklyn Center', 'Coon Rapids', 'Shoreview', 'Roseville', 'Fridley', 'Blaine', 'Lino Lakes', 'Arden Hills', 'Columbia Heights', 'Anoka', 'Mounds View', 'New Hope', 'North Oaks', 'Little Canada', 'Dellwood', 'Mahtomedi', 'Hilltop', 'Lauderdale', 'Golden Valley', 'Dayton', 'Rogers', 'White Bear Lake', 'Edina', 'Vadnais Heights', 'Mendota', 'Maplewood', 'Falcon Heights', 'Stillwater', 'No

Simulation45 for Year2021:['Minneapolis', 'Hilltop', 'Mendota', 'Columbia Heights', 'Lilydale', 'Newport', 'Medicine Lake', 'West Saint Paul', 'Shoreview', 'Vadnais Heights', 'Mendota Heights', 'New Hope', 'Inver Grove Heights', 'Maple Grove', 'Hopkins', 'Saint Paul Park', 'Coates', 'Lino Lakes', 'Birchwood Village', 'Mahtomedi', 'North Saint Paul', 'White Bear Lake', 'Brooklyn Park', 'Landfall', 'Blaine', 'Mounds View', 'Ham Lake', 'Corcoran', 'Crystal', 'Brooklyn Center', 'Eagan', 'Saint Paul', 'Arden Hills', 'Lauderdale', 'Grant', 'Woodbury', 'Maplewood', 'Sunfish Lake', 'Cottage Grove', 'Roseville', 'Falcon Heights', 'Spring Lake Park', 'Golden Valley', 'Rockford', 'New Brighton', 'Wayzata', 'Richfield', 'South Saint Paul', 'Stillwater', 'Bloomington', 'Fridley', 'Little Canada', 'Oakdale', 'Lexington', 'Minnetonka Beach', 'Spring Park', 'Centerville', 'Lake Saint Croix Beach', 'Apple Valley', 'Dellwood', 'Lake Elmo', 'East Bethel', 'Greenwood', 'Lakeland Shores', 'North Oaks', 'Ge

In [39]:
#get the simulation final prediction
arcpy.management.AddField("City_Sim", f"Sim_MC", "SHORT")
arcpy.management.AddField("City_Pred2022", f"Pred_MC", "SHORT")
arcpy.management.AddField("City_Pred2023", f"Pred_MC", "SHORT")
arcpy.management.AddField("City_Pred2024", f"Pred_MC", "SHORT")

final_result("City_Sim",method = 'MC')        
#delete unnecessary columns
arcpy.management.DeleteField("City_Sim", "CITY_NAME;POPULATION;Sim_MC", "KEEP_FIELDS")

final_result("City_Pred2022",method = 'MC')
#delete unnecessary columns
arcpy.management.DeleteField("City_Pred2022", "CITY_NAME;POPULATION;Pred_MC", "KEEP_FIELDS")

final_result("City_Pred2023",method = 'MC')
#delete unnecessary columns
arcpy.management.DeleteField("City_Pred2023", "CITY_NAME;POPULATION;Pred_MC", "KEEP_FIELDS")

final_result("City_Pred2024",method = 'MC')
#delete unnecessary columns
arcpy.management.DeleteField("City_Pred2024", "CITY_NAME;POPULATION;Pred_MC", "KEEP_FIELDS")

<Result 'City_Pred2024'>

### Gravity Model - Pop & Dist

In [40]:
# adds probability column to near table
arcpy.management.AddField("City_FromToTable_Final", "Probability_G", "DOUBLE")

#Calculates probability of each pair transition
alpha = 1.5

with arcpy.da.SearchCursor("City_FromToTable_Final","*") as cursor:
    total_sum = sum(((row[5] * row[7])/(row[3]**alpha)) for row in cursor)
with arcpy.da.UpdateCursor("City_FromToTable_Final","*") as cursor:
    for row in cursor:
        row[10] = (row[5] * row[7])/(row[3]**alpha)/total_sum
        cursor.updateRow(row)
del cursor

In [41]:
#run 50 simulations
for i in range(0,50):
    PresenceCity = simulation("City_FromToTable_Final","City_Sim",method = 'G',
                              seed = 'Minneapolis',num_year = 6,G = 30,i = i)
    
    PresenceCity = prediction("City_FromToTable_Final","City_Pred2022",method = 'G',PresenceCity = PresenceCity, G = 30,i = i)
    PresenceCity = prediction("City_FromToTable_Final","City_Pred2023",method = 'G',PresenceCity = PresenceCity, G = 30,i = i)
    PresenceCity = prediction("City_FromToTable_Final","City_Pred2024",method = 'G',PresenceCity = PresenceCity, G = 30,i = i)

Simulation0 for Year2021:['Minneapolis', 'Saint Paul', 'Plymouth', 'Blaine', 'Coon Rapids', 'Rockford', 'Owatonna', 'Crystal', 'Richfield', 'Oakdale', 'Golden Valley', 'Waconia', 'New Hope', 'Lakeville', 'Stillwater', 'Columbia Heights', 'Savage', 'Kerkhoven', 'Brooklyn Center', 'Bloomington', 'Mounds View', 'South Saint Paul', 'Burnsville', 'Inver Grove Heights', 'Roseville', 'Saint Michael', 'Edina', 'Fridley', 'Minnetonka', 'New Brighton', 'Elk River', 'Saint Louis Park', 'West Saint Paul', 'Brooklyn Park', 'Oak Park Heights', 'North Saint Paul', 'Woodbury', 'Shoreview', 'Annandale', 'Deephaven', 'Chanhassen', 'Austin', 'Maple Grove', 'Oak Grove']
Simulation1 for Year2021:['Minneapolis', 'Edina', 'Saint Paul', 'Eden Prairie', 'Sauk Rapids', 'Maplewood', 'Eagan', 'Fridley', 'Bloomington', 'Chaska', 'North Saint Paul', 'Apple Valley', 'Burnsville', 'Lexington', 'Rosemount', 'Brooklyn Center', 'Roseville', 'Savage', 'Shakopee', 'Coon Rapids', 'Northfield', 'Hastings', 'Champlin', 'Lino

Simulation26 for Year2021:['Minneapolis', 'Falcon Heights', 'Saint Louis Park', 'Robbinsdale', 'Roseville', 'Shoreview', 'Brooklyn Park', 'Saint Paul Park', 'Shakopee', 'Eden Prairie', 'Saint Paul', 'Lake Elmo', 'Burnsville', 'Andover', 'Eagan', 'Bloomington', 'Woodbury', 'Saint Francis', 'Fridley', 'Cottage Grove', 'Maple Grove', 'Rosemount', 'New Hope', 'Edina', 'Saint Michael', 'Eagle Lake', 'Lakeville', 'West Saint Paul', 'Chanhassen', 'Inver Grove Heights', 'Osseo', 'North Oaks', 'New Brighton', 'Plymouth', 'Coon Rapids', 'North Saint Paul', 'Hopkins', 'Mound', 'Mayer', 'Apple Valley', 'Crystal', 'Red Wing', 'Savage', 'Champlin', 'Rogers', 'Columbia Heights', 'Richfield', 'Stillwater', 'Saint Cloud', 'Saint Joseph', 'Brooklyn Center', 'Albertville']
Simulation27 for Year2021:['Minneapolis', 'Saint Louis Park', 'Brooklyn Center', 'Fridley', 'Bloomington', 'Eden Prairie', 'Anoka', 'Golden Valley', 'Edina', 'Saint Paul', 'Minnetonka', 'Coon Rapids', 'Prior Lake', 'Northfield', 'Apple

In [42]:
#get the simulation final prediction
arcpy.management.AddField("City_Sim", f"Sim_G", "SHORT")
arcpy.management.AddField("City_Pred2022", f"Pred_G", "SHORT")
arcpy.management.AddField("City_Pred2023", f"Pred_G", "SHORT")
arcpy.management.AddField("City_Pred2024", f"Pred_G", "SHORT")

final_result("City_Sim",method = 'G')        
#delete unnecessary columns
arcpy.management.DeleteField("City_Sim", "CITY_NAME;POPULATION;Sim_MC;Sim_G", "KEEP_FIELDS")

final_result("City_Pred2022",method = 'G')
#delete unnecessary columns
arcpy.management.DeleteField("City_Pred2022", "CITY_NAME;POPULATION;Pred_MC;Pred_G", "KEEP_FIELDS")

final_result("City_Pred2023",method = 'G')
#delete unnecessary columns
arcpy.management.DeleteField("City_Pred2023", "CITY_NAME;POPULATION;Pred_MC;Pred_G", "KEEP_FIELDS")

final_result("City_Pred2024",method = 'G')
#delete unnecessary columns
arcpy.management.DeleteField("City_Pred2024", "CITY_NAME;POPULATION;Pred_MC;Pred_G", "KEEP_FIELDS")

<Result 'City_Pred2024'>

### Huff Model - LandUse, Pop, Dist

In [43]:
# adds probability column to near table
arcpy.management.AddField("City_FromToTable_Final", "Probability_H", "DOUBLE")

#Calculates probability of each pair transition
alpha = 1.5

with arcpy.da.SearchCursor("City_FromToTable_Final","*") as cursor:
    total_sum = sum(((row[7] * (row[9]+0.001))/(row[3]**alpha)) for row in cursor)
with arcpy.da.UpdateCursor("City_FromToTable_Final","*") as cursor:
    for row in cursor:
        row[11] = ((row[7] * (row[9]+0.001))/(row[3]**alpha))/total_sum
        cursor.updateRow(row)
del cursor

In [44]:
#run 50 simulations
for i in range(0,50):
    
    PresenceCity = simulation("City_FromToTable_Final","City_Sim",method = 'H',
                              seed = 'Minneapolis',num_year = 6,G = 250,i = i)
    
    PresenceCity = prediction("City_FromToTable_Final","City_Pred2022",method = 'H',PresenceCity = PresenceCity, G = 250,i = i)
    PresenceCity = prediction("City_FromToTable_Final","City_Pred2023",method = 'H',PresenceCity = PresenceCity, G = 250,i = i)
    PresenceCity = prediction("City_FromToTable_Final","City_Pred2024",method = 'H',PresenceCity = PresenceCity, G = 250,i = i)

Simulation0 for Year2021:['Minneapolis', 'Brooklyn Center', 'Eden Prairie', 'Red Wing', 'Shakopee', 'Cottage Grove', 'Brooklyn Park', 'Orono', 'North Branch', 'Arden Hills', 'Ham Lake', 'Blaine', 'Mendota Heights', 'Minnetonka', 'Shoreview', 'New Brighton', 'Coon Rapids', 'Saint Michael', 'Champlin', 'Andover', 'Hugo', 'Chanhassen', 'Lakeville', 'Circle Pines', 'Lake Elmo', 'Farmington', 'Rogers', 'Bloomington', 'North Oaks', 'Saint Cloud', 'Saint Paul', 'Maplewood', 'Robbinsdale', 'New Prague', 'Woodbury', 'Edina', 'Shorewood', 'Eagan', 'Elk River', 'Maple Grove', 'Deephaven', 'Prior Lake', 'East Bethel', 'Medina', 'Savage', 'Ramsey', 'Saint Louis Park', 'Plymouth', 'Otsego', 'Inver Grove Heights', 'Burnsville', 'Duluth', 'Vadnais Heights', 'Newport', 'Lonsdale', 'Mound', 'Waconia', 'Minnetrista', 'Credit River', 'Wayzata', 'Chaska', 'Delano', 'Saint Paul Park', 'Roseville']
Simulation1 for Year2021:['Minneapolis', 'Brooklyn Center', 'Woodbury', 'Byron', 'North Oaks', 'Lino Lakes', 'S

Simulation19 for Year2021:['Minneapolis', 'Saint Paul', 'Eden Prairie', 'Shoreview', 'Inver Grove Heights', 'Woodbury', 'Arden Hills', 'Kasson', 'Blaine', 'Richfield', 'Bloomington', 'Shorewood', 'Maple Grove', 'Forest Lake', 'West Saint Paul', 'Chanhassen', 'Apple Valley', 'Fridley', 'Rosemount', 'Maple Lake', 'Ramsey', 'Burnsville', 'Savage', 'Anoka', 'Eagan', 'Lindstrom', 'Lakeville', 'Pine Springs', 'Cottage Grove', 'Plymouth', 'Maplewood', 'Red Wing', 'Farmington', 'Corcoran', 'Little Canada', 'Ham Lake', 'Greenfield', 'Andover', 'Elk River', 'Victoria', 'Becker', 'Hastings', 'Brooklyn Park', 'Roseville', 'Nowthen', 'New Ulm', 'Orono', 'Minnetrista', 'Duluth', 'Rockford', 'Vadnais Heights', 'White Bear Lake', 'North Branch', 'Edina', 'Chaska', 'Owatonna', 'Oak Grove', 'Saint Anthony', 'Credit River', 'North Oaks', 'Scandia', 'Minnetonka', 'Waconia']
Simulation20 for Year2021:['Minneapolis', 'Saint Paul', 'Woodbury', 'Maplewood', 'Brooklyn Park', 'Elk River', 'Maple Grove', 'Shorev

Simulation40 for Year2021:['Minneapolis', 'Robbinsdale', 'Chanhassen', 'Dayton', 'Shorewood', 'Rogers', 'Ramsey', 'Otsego', 'Victoria', 'Credit River', 'Albertville', 'Maple Grove', 'Big Lake', 'Austin', 'Waite Park', 'Chaska', 'Shakopee', 'Blaine', 'Champlin', 'Mayer', 'Bloomington', 'Eagan', 'Mendota Heights', 'Lakeville', 'Eden Prairie', 'Ham Lake', 'Elk River', 'Saint Michael', 'Minnetrista', 'Cottage Grove', 'Lakeland', 'Red Wing', 'Brooklyn Park', 'Burnsville', 'Hastings', 'Hugo', 'Chisago City', 'Centerville', 'Lakeland Shores', 'Maplewood', 'Cambridge', 'Mahtomedi', 'Woodbury', 'Lindstrom', 'Saint Paul', 'Plymouth', 'Saint Cloud', 'North Saint Paul', 'Savage', 'Minnetonka', 'Prior Lake', 'Elko New Market', 'Lino Lakes', 'Coon Rapids', 'Faribault', 'Anoka', 'Inver Grove Heights', 'Apple Valley', 'Saint Louis Park', 'Orono', 'Buffalo', 'Mankato']
Simulation41 for Year2021:['Minneapolis', 'Saint Paul', 'Falcon Heights', 'Newport', 'Victoria', 'Carver', 'Lino Lakes', 'Roseville', '

In [45]:
#get the simulation final prediction
arcpy.management.AddField("City_Sim", f"Sim_H", "SHORT")
arcpy.management.AddField("City_Pred2022", f"Pred_H", "SHORT")
arcpy.management.AddField("City_Pred2023", f"Pred_H", "SHORT")
arcpy.management.AddField("City_Pred2024", f"Pred_H", "SHORT")

final_result("City_Sim",method = 'H')        
#delete unnecessary columns
arcpy.management.DeleteField("City_Sim", "CITY_NAME;POPULATION;Sim_MC;Sim_G;Sim_H", "KEEP_FIELDS")

final_result("City_Pred2022",method = 'H')
#delete unnecessary columns
arcpy.management.DeleteField("City_Pred2022", "CITY_NAME;POPULATION;Pred_MC;Pred_G;Pred_H", "KEEP_FIELDS")

final_result("City_Pred2023",method = 'H')
#delete unnecessary columns
arcpy.management.DeleteField("City_Pred2023", "CITY_NAME;POPULATION;Pred_MC;Pred_G;Pred_H", "KEEP_FIELDS")

final_result("City_Pred2024",method = 'H')
#delete unnecessary columns
arcpy.management.DeleteField("City_Pred2024", "CITY_NAME;POPULATION;Pred_MC;Pred_G;Pred_H", "KEEP_FIELDS")

<Result 'City_Pred2024'>

### Assessment comparision for three models

In [2]:
#Spatial Join BMSB with CITYNAME AND POPULATION for further assessment
#arcpy.SpatialJoin_analysis('bmsb', "City_qaqc_done", 'bmsb_SpatialJoin',join_operation="JOIN_ONE_TO_ONE", match_option = "CLOSEST_GEODESIC")

# get true presence cities
bmsb_city = []
with arcpy.da.SearchCursor("bmsb_SpatialJoin", ["CITY_NAME"]) as cursor:
    for row in cursor:
        if row[0] is not None:
            bmsb_city.append(row[0])
            
bmsb_city = list(set(bmsb_city))

In [3]:
#confusion matrix
def get_ConfusionMatrix(pred,name,truth,TP,TN,FP,FN):
    if (pred == 1) and (name in truth):
        TP +=1
    elif (pred == 0) and (name not in truth):
        TN +=1
    elif (pred == 1) and (name not in truth):
        FP +=1
    elif (pred == 0) and (name in truth):
        FN +=1
    return TP,TN,FP,FN

TP_G,TN_G,FP_G,FN_G = 0,0,0,0
TP_MC,TN_MC,FP_MC,FN_MC = 0,0,0,0
TP_H,TN_H,FP_H,FN_H = 0,0,0,0

with arcpy.da.SearchCursor("City_Sim", ["CITY_NAME","Sim_G",'Sim_MC','Sim_H']) as cursor:
    for row in cursor:
        TP_G,TN_G,FP_G,FN_G = get_ConfusionMatrix(row[1],row[0],bmsb_city,TP_G,TN_G,FP_G,FN_G)
        TP_MC,TN_MC,FP_MC,FN_MC = get_ConfusionMatrix(row[2],row[0],bmsb_city,TP_MC,TN_MC,FP_MC,FN_MC)
        TP_H,TN_H,FP_H,FN_H = get_ConfusionMatrix(row[3],row[0],bmsb_city,TP_H,TN_H,FP_H,FN_H)

In [4]:
#accuracy
accuracy_MC =  (TP_MC + TN_MC) / (TP_MC + TN_MC + FP_MC + FN_MC)
accuracy_G =  (TP_G + TN_G) / (TP_G + TN_G + FP_G + FN_G)
accuracy_H =  (TP_H + TN_H) / (TP_H + TN_H + FP_H + FN_H)
print(f"The accuracy for monte carlo model is:{accuracy_MC}")
print(f"The accuracy for gravity model is:{accuracy_G}")
print(f"The accuracy for huff model is:{accuracy_H}")

#precision
precision_MC = TP_MC / (TP_MC + FP_MC)
precision_G = TP_G / (TP_G + FP_G)
precision_H = TP_H / (TP_H + FP_H)
print(f"The precision for monte carlo model is:{precision_MC}")
print(f"The precision for gravity model is:{precision_G}")
print(f"The precision for huff model is:{precision_H}")

#recall
recall_MC = TP_MC / (TP_MC + FN_MC)
recall_G = TP_G / (TP_G + FN_G)
recall_H = TP_H / (TP_H + FN_H)
print(f"The recall for monte carlo model is:{recall_MC}")
print(f"The recall for gravity model is:{recall_G}")
print(f"The recall for huff model is:{recall_H}")

The accuracy for monte carlo model is:0.9297423887587822
The accuracy for gravity model is:0.9437939110070258
The accuracy for huff model is:0.9508196721311475
The precision for monte carlo model is:0.1951219512195122
The precision for gravity model is:0.3142857142857143
The precision for huff model is:0.40540540540540543
The recall for monte carlo model is:0.22857142857142856
The recall for gravity model is:0.3142857142857143
The recall for huff model is:0.42857142857142855


### Get the priority monitoring city

In [10]:
#first priority cities (FP in 2021)
# get true presence cities
city2021 = []
with arcpy.da.SearchCursor("City_Sim", ["CITY_NAME",'Sim_H']) as cursor:
    for row in cursor:
        if row[1] == 1 and row[0] not in bmsb_city:
            city2021.append(row[0])

#second priority cities (will spread in 2022)
city2022 = []
with arcpy.da.SearchCursor("City_Pred2022", ["CITY_NAME",'Pred_H']) as cursor:
    for row in cursor:
        if row[1] == 1 and row[0] not in bmsb_city and row[0] not in city2021:
            city2022.append(row[0])

#third priority cities (will spread in 2023)
city2023 = []
with arcpy.da.SearchCursor("City_Pred2023", ["CITY_NAME",'Pred_H']) as cursor:
    for row in cursor:
        if row[1] == 1 and row[0] not in bmsb_city and row[0] not in city2021 and row[0] not in city2022:
            city2023.append(row[0])
            
#forth priority cities (will spread in 2024)
city2024 = []
with arcpy.da.SearchCursor("City_Pred2024", ["CITY_NAME",'Pred_H']) as cursor:
    for row in cursor:
        if row[1] == 1 and row[0] not in bmsb_city and row[0] not in city2021 and row[0] not in city2022 and row[0] not in city2023:
            city2024.append(row[0])

In [18]:
arcpy.management.AddField("City_Point", f"Priority", "SHORT")
with arcpy.da.UpdateCursor("City_Point", ["CITY_NAME",'Priority']) as cursor:
    for row in cursor:
        if row[0] in city2021:
            row[1] = 5
        elif row[0] in city2022:
            row[1] = 4
        elif row[0] in city2023:
            row[1] = 3
        elif row[0] in city2024:
            row[1] = 2
        else:
            row[1] = 1
        cursor.updateRow(row)

In [19]:
arcpy.management.AddField("City_qaqc_done", f"Priority", "SHORT")
with arcpy.da.UpdateCursor("City_qaqc_done", ["CITY_NAME",'Priority']) as cursor:
    for row in cursor:
        if row[0] in city2021:
            row[1] = 5
        elif row[0] in city2022:
            row[1] = 4
        elif row[0] in city2023:
            row[1] = 3
        elif row[0] in city2024:
            row[1] = 2
        else:
            row[1] = 1
        cursor.updateRow(row)